In [1]:
!pip install influxdb-client

In [25]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
from urllib3.exceptions import NewConnectionError
from influxdb_client import Point

INFLUX_URL = "http://influxdb2:8086"
INFLUX_TOKEN = "MyInitialAdminToken0="

print("--- Iniciando conexión a InfluxDB ---")

client = None
try:
    # 1. Inicializar el cliente
    client = influxdb_client.InfluxDBClient(
        url=INFLUX_URL,
        token=INFLUX_TOKEN,
        org="docs"
    )

    # 2. Verificar la conexión con el servidor
    print(f"Verificando estado de salud de InfluxDB en {INFLUX_URL}...")
    health = client.health()

    if health.status == "pass":
        print("[INFO] ¡Conexión exitosa!")
        print(f"[INFO] Versión del servidor: {health.version}")
    else:
        print(f"[ERROR] Conexión fallida. Estado: {health.status}")
        print(f"[INFO] Mensaje: {health.message}")

except (InfluxDBError, NewConnectionError) as e:
    print("[ERROR] Error al conectar con InfluxDB:")
    print(f"    Detalle: {e}")

#finally:
    # 3. Es buena práctica cerrar siempre el cliente
#    if client:
 #       client.close()
  #      print("--- Conexión cerrada ---")

--- Iniciando conexión a InfluxDB ---
Verificando estado de salud de InfluxDB en http://influxdb2:8086...
[INFO] ¡Conexión exitosa!
[INFO] Versión del servidor: v2.7.12


In [26]:
write_api = client.write_api(write_options=SYNCHRONOUS)
p = Point("temperatura_mediacion").tag("location", "oficina").field("value", 23.5).time(None)

write_api.write(bucket="pruebas", record=p)

In [29]:
import datetime

tiempo_actual= datetime.datetime.utcnow().isoformat() + "Z"

punto = Point("datos_iot").tag("ubicacion", "bodega_sur").tag("sensor_id", "S003").field("humedad relativa", 75.8).field("nivel_bateria", 95).field("activo", True).time(tiempo_actual)

write_api.write(bucket="pruebas", record=punto)

In [34]:
query_api = client.query_api()

query_ultimo_dia = f"""
from (bucket: "pruebas")
    |> range(start: -1d)
"""

result= query_api.query(query= query_ultimo_dia)

In [37]:
print("--- Imprimiendo resultados ---")

for table in result:
    for record in table.records:
        # Usamos .values.get() para obtener la ubicación de forma segura
        # Si no existe, pondrá "N/A"
        ubicacion = record.values.get("location", "N/A")
        valor = record.get_value()
        hora = record.get_time().strftime('%H:%M:%S')
        
        print(f"[{hora}] Ubicación: {ubicacion} | Valor: {valor}")

--- Imprimiendo resultados ---
[09:38:44] Ubicación: N/A | Valor: True
[09:38:44] Ubicación: N/A | Valor: 75.8
[09:38:44] Ubicación: N/A | Valor: 95
[09:34:32] Ubicación: oficina | Valor: 23.5
